In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class Net(nn.Module):
    
    def  __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5X5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        #an affine operation: y = Wx+b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square, one can specify only a single digit
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimentions except the batch dimention
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    

net = Net()
print(net)
        
        
        

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


we just need to define forward function; the backward function is automatically defined using `autograd`

In [9]:
## the learnable parameters of a model is return by net.parameters()
params = list(net.parameters())
print(len(params))
print("conv1's weight  {}".format(params[0].size()))

10
conv1's weight  torch.Size([6, 1, 5, 5])


In [10]:
## the expected input size of this network is 32x32
# we wil resize the image for this
input = Variable(torch.randn(1,1,32,32))
out = net(input)
print(out)

Variable containing:
 0.0528 -0.0869 -0.0999  0.0922  0.0630 -0.0041 -0.1810  0.1295 -0.0317  0.0257
[torch.FloatTensor of size 1x10]



In [11]:
# zero the gradient buffer of all parameters
net.zero_grad()

# backprop with random gradients
out.backward(torch.randn(1,10))

In [15]:
output = net(input)
target = Variable(torch.arange(1,11))
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

Variable containing:
 38.5127
[torch.FloatTensor of size 1]



In [20]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [21]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  1.2894
 -4.9328
  2.5536
  2.4162
  7.1702
  6.2629
[torch.FloatTensor of size 6]



In [22]:
## Updating weight by writing simple python code
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [23]:
## using pytorch functionality 
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update